In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import random

In [38]:
ndb = pd.read_csv('zip_data.csv')
ndb.head()

,ZIP,TOTAL,RESIDENTS
0,10001,13870,275
1,10002,34748,271
2,10003,33932,408
3,10004,2075,80
4,10005,6745,111


In [39]:
cdb = pd.read_csv('company_data.csv')
cdb.head()

,ZIP,EMPLOYEES
0,10022,9000
1,10011,6775
2,10001,4410
3,10007,1400
4,10003,1400


In [40]:
cdbz = cdb.groupby('ZIP').sum().reset_index()
cdbz.head()

,ZIP,EMPLOYEES
0,10001,6161
1,10003,3203
2,10007,2142
3,10010,2090
4,10011,7558


In [41]:
wdf = pd.merge(cdbz, ndb, on='ZIP', how='outer').reset_index()
wdf.head()

,index,ZIP,EMPLOYEES,TOTAL,RESIDENTS
0,0,10001,6161.0,13870.0,275.0
1,1,10003,3203.0,33932.0,408.0
2,2,10007,2142.0,3738.0,8.0
3,3,10010,2090.0,20291.0,350.0
4,4,10011,7558.0,33421.0,412.0


In [16]:
wdf['EMPLOYEES'].fillna(0, inplace=True)
wdf['RESIDENTS'].fillna(0, inplace=True)
wdf['EMPLOYEES'] = wdf['EMPLOYEES'] * .272 // 1
wdf['TOTAL'] = wdf['EMPLOYEES'] + wdf['RESIDENTS']
wdf=wdf[['ZIP','TOTAL']]
wdf.shape

(214, 2)

In [42]:
sdf = pd.read_csv('top_zip_stations.csv')
sdf.head()

,ZIP,STATION
0,10003,1 AV
1,10011,14 ST
2,10003,14 ST-UNION SQ
3,10011,18 ST
4,10010,23 ST


In [43]:
tsdf = pd.merge(sdf,wdf,how='inner',on='ZIP').reset_index()
tsdf['TECH_TOTAL'] = tsdf['TOTAL']
tsdf = tsdf[['ZIP','TECH_TOTAL','STATION']]

tsdf.head()

,ZIP,TECH_TOTAL,STATION
0,10003,33932.0,1 AV
1,10003,33932.0,14 ST-UNION SQ
2,10003,33932.0,3 AV
3,10003,33932.0,8 ST-NYU
4,10003,33932.0,ASTOR PL


In [44]:
stations_mta =[
    '34 ST-PENN STA',
    'GRD CNTRL-42 ST',
    '34 ST-HERALD SQ',
    '23 ST',
    '14 ST-UNION SQ',
    'TIMES SQ-42 ST',
    'FULTON ST',
    '86 ST',
    '42 ST-PORT AUTH',
    '59 ST COLUMBUS',
    ]

In [45]:
mta_data = pd.read_csv('../notebooks/output_stage2.csv')
mta_data['MTA_TOTALS'] = mta_data['ENTRY_DIFF']+mta_data['EXIT_DIFF']
mta_data = mta_data.loc[:,['STATION','DATE','MTA_TOTALS',]] 
mta_data= mta_data.groupby(['STATION','DATE']).sum().reset_index()
mta_data = mta_data.groupby('STATION').mean().reset_index()
mta_data.head()

,STATION,MTA_TOTALS
0,1 AV,41752.351648
1,103 ST,47437.725275
2,103 ST-CORONA,30471.054945
3,104 ST,3621.263736
4,110 ST,17537.208791


In [21]:
mta_select = mta_data.loc[mta_data['STATION'].isin(stations_mta),:].reset_index()
mta_select

,index,STATION,MTA_TOTALS
0,9,125 ST,121082.483516
1,14,14 ST-UNION SQ,173524.967033
2,45,23 ST,158378.252747
3,58,34 ST-HERALD SQ,184084.439560
4,60,34 ST-PENN STA,262622.076923
5,67,42 ST-PORT AUTH,144134.021978
6,85,59 ST COLUMBUS,112928.472527
7,109,86 ST,127295.285714
8,225,FULTON ST,150683.318681
9,232,GRD CNTRL-42 ST,223169.912088


In [46]:
merge_data = pd.merge(mta_data, tsdf, on='STATION',how='inner').reset_index()

#merge_data =merge_data.groupby(['STATION','ZIP']).sum().reset_index()


In [47]:
merge_data = merge_data.loc[merge_data['STATION'].isin (stations_mta),:]
merge_data

,index,STATION,MTA_TOTALS,ZIP,TECH_TOTAL
2,2,14 ST-UNION SQ,173524.967033,10003,33932.0
4,4,23 ST,158378.252747,10011,33421.0
5,5,23 ST,158378.252747,10010,20291.0
10,10,34 ST-HERALD SQ,184084.439560,10001,13870.0
12,12,34 ST-PENN STA,262622.076923,10001,13870.0
13,13,42 ST-PORT AUTH,144134.021978,10036,18641.0
23,23,86 ST,127295.285714,10024,34519.0
24,24,86 ST,127295.285714,10024,34519.0
26,26,GRD CNTRL-42 ST,223169.912088,10016,32969.0
28,28,TIMES SQ-42 ST,174196.153846,10036,18641.0


In [ ]:
# plt.figure(figsize=(12,8))
# y_cor = list((merge_data['MTA_TOTALS']))
# x_cor = list((merge_data['TECH_TOTAL']))
# stat =list(merge_data['STATION'])
# plt.scatter(merge_data['MTA_TOTALS'],merge_data['TECH_TOTAL'])

# for i in range(len(y_cor)):
#     plt.text(int(y_cor[i]),int(x_cor[i])+random.randint(-50,50), stat[i],fontsize=9)
    